In [24]:
import pandas as pd
import pprint
import ast
import json
import numpy as np
from scipy.stats import entropy
from scipy.special import softmax
import numpy as np
import pandas as pd
from scipy.special import softmax
from scipy.stats import entropy
import ast



In [25]:
from dict import news_labels, yahoo_labels, newsgroups_labels, depression_labels, emotion_labels, rating_labels, depression, newsgroups, go_emotions, ag_news, yahoo, sst5

In [26]:
model = 'gpt4omini'
dataset_tag = 'sst5'

In [27]:
dict_map = rating_labels
N = sst5['num_classes'] * sst5['num_samples']
print(N)
selected_method = f'entropy_{N}'

60


In [28]:

prompt = 'no_samples_numbers'
temp = 0
exp = 1

In [29]:
selcted_samples = ''

In [30]:
path = f'../NO_EXAMPLES/results/{prompt}_temp{0}/{model}/{dataset_tag}_{model}_{prompt}_temp{temp}_exp{exp}.csv'

In [31]:
import os
output_path = f'SELECTED_SAMPLES/{dataset_tag}/{model}_{selected_method}.csv'
folder = os.path.dirname(output_path)
os.makedirs(folder, exist_ok=True)

In [32]:

def calculate_metric_with_true_label(df, column_name):
    

    def parse_top_logprobs(val):
        if isinstance(val, str):
            try:
                return ast.literal_eval(val)
            except Exception as e:
                print(f"❌ Nie udało się sparsować: {val[:60]}... ➡ {e}")
                return None
        return val

    df['top_logprobs'] = df['top_logprobs'].apply(parse_top_logprobs)
    
    df['logprobs_cal'] = df['top_logprobs'].apply(
    lambda x: np.array([item['logprob'] for item in x if isinstance(item, dict) and 'logprob' in item])
    if isinstance(x, list) else np.array([])
)



    # Przekształcenie logprobs do prawdopodobieństw przez softmax
    df['probs'] = df['logprobs_cal'].apply(
        lambda logprobs: softmax(logprobs) if len(logprobs) > 0 else np.array([])
    )

    # Entropia rozkładu
    df['entropy'] = df['probs'].apply(
        lambda probs: entropy(probs, base=2) if len(probs) > 0 else np.nan
    )

    # Margin: różnica między top-1 a top-2
    df['margin'] = df['probs'].apply(
        lambda probs: np.sort(probs)[-1] - np.sort(probs)[-2] if len(probs) > 1 else np.nan
    )

    # KL divergence względem idealnego rozkładu (1.0 na poprawnej etykiecie)
    def kl_div(probs, top_logprobs, label):
        if not isinstance(top_logprobs, list) or len(probs) == 0:
            return np.nan

        try:
            # lista tokenów jako stringi
            tokens = [str(item['token']).strip().lower() for item in top_logprobs if isinstance(item, dict) and 'token' in item]
            label_str = str(label).strip().lower()

            if label_str not in tokens:
                return np.nan

            match_idx = tokens.index(label_str)
            ideal_dist = np.zeros_like(probs)
            ideal_dist[match_idx] = 1.0

            return entropy(ideal_dist, probs, base=2)

        except Exception as e:
            print(f"⚠️ Błąd w kl_div: {e}")
            return np.nan


    # df['kl_divergence'] = df.apply(
    #     lambda row: kl_div(row['probs'], row['top_logprobs'], row[column_name]), axis=1
    # )
    
    df['kl_divergence'] = df['probs'].apply(
    lambda probs: np.sum(probs * np.log(probs / (1 / len(probs)))) if len(probs) > 0 else np.nan
        )
    
    # df['top_prob'] = df['probs'].apply(lambda p: np.max(p) if len(p) > 0 else np.nan)
    # df['kl_divergence'] = 2 * (1 - df['top_prob'])



    # Normalizacja (min-max)
    for col in ['entropy', 'margin', 'kl_divergence']:
        norm_col = f'normalized_{col}'
        min_val = df[col].min()
        max_val = df[col].max()
        if pd.notnull(min_val) and pd.notnull(max_val) and max_val > min_val:
            df[norm_col] = (df[col] - min_val) / (max_val - min_val)
        else:
            df[norm_col] = 0.0

    return df




In [33]:


df = pd.read_csv(path)


In [34]:
df['original_label_map'] = df['original_label'].map(dict_map)

In [35]:
df

,text,output,logprobs,top_logprobs,original_label,original_label_map
0,Though the opera itself takes place mostly ind...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",3,3
1,If you can read the subtitles ( the opera is s...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.0...",4,4
2,( `` Safe Conduct '' ) is a long movie at 163 ...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.0...",4,4
3,The beauty of Alexander Payne 's ode to the Ev...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.0...",4,4
4,The fourth in a series that I 'll bet most par...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.6...",3,3
...,...,...,...,...,...,...
995,"In other words , it 's badder than bad .",2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.5...",1,1
996,The direction occasionally rises to the level ...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -2.1...",2,2
997,The movie is ingenious fun .,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.3...",5,5
998,The cast comes through even when the movie doe...,3,"{'content': [{'token': '3', 'bytes': [51], 'lo...","[{'token': '3', 'bytes': [51], 'logprob': -0.1...",3,3


In [36]:
# import re
# import numpy as np

# def extract_number(text):
#     match = re.search(r'\b\d+\b', str(text))
#     if match:
#         return int(match.group())
#     else:
#         return np.nan  # lub inna wartość domyślna

# df['output'] = df['output'].apply(extract_number)
# df['original_label_map'] = df['original_label_map'].apply(extract_number)

# df['output'] = df['output'].astype('int64')
# df['original_label_map'] = df['original_label_map'].astype('int64')



In [37]:
print(df['original_label_map'].isnull().sum())

0


In [38]:
df_metric = calculate_metric_with_true_label(df, 'original_label_map')
df_metric

,text,output,logprobs,top_logprobs,original_label,original_label_map,logprobs_cal,probs,entropy,margin,kl_divergence,normalized_entropy,normalized_margin,normalized_kl_divergence
0,Though the opera itself takes place mostly ind...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.0...",3,3,"[-0.00861456897109747, -4.758614540100098, -18...","[0.9914225014113162, 0.008577485547366722, 1.0...",0.071208,0.982845,2.946374,0.052440,0.982845,0.947560
1,If you can read the subtitles ( the opera is s...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.0...",4,4,"[-0.01416350994259119, -4.264163494110107, -17...","[0.9859363200772531, 0.014063626511622032, 1.9...",0.106666,0.971873,2.921797,0.078551,0.971873,0.921449
2,( `` Safe Conduct '' ) is a long movie at 163 ...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.0...",4,4,"[-0.00017982252757064998, -8.6251802444458, -1...","[0.999820256138243, 0.0001795278548350011, 1.8...",0.002498,0.999641,2.994001,0.001840,0.999641,0.998160
3,The beauty of Alexander Payne 's ode to the Ev...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.0...",4,4,"[-0.03804149478673935, -3.288041591644287, -17...","[0.9626730751864853, 0.03732688227589327, 3.98...",0.229900,0.925346,2.836378,0.169305,0.925346,0.830695
4,The fourth in a series that I 'll bet most par...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.6...",3,3,"[-0.6931512355804443, -0.6931512355804443, -12...","[0.4999979641064766, 0.4999979641064766, 3.072...",1.000078,0.000000,2.302531,0.736486,0.000000,0.263514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"In other words , it 's badder than bad .",2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.5...",1,1,"[-0.5759444236755371, -0.8259444236755371, -12...","[0.5621736740160835, 0.4378212975458545, 5.025...",0.988907,0.124352,2.310274,0.728259,0.124352,0.271741
996,The direction occasionally rises to the level ...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -2.1...",2,2,"[-2.1008713702030946e-06, -13.125001907348633,...","[0.9999979505788533, 1.9947299984578005e-06, 5...",0.000042,0.999996,2.995703,0.000031,0.999996,0.999969
997,The movie is ingenious fun .,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.3...",5,5,"[-0.3868710398674011, -1.136871099472046, -19....","[0.6791787051558535, 0.3208212845271373, 2.308...",0.905273,0.358357,2.368245,0.666668,0.358357,0.333332
998,The cast comes through even when the movie doe...,3,"{'content': [{'token': '3', 'bytes': [51], 'lo...","[{'token': '3', 'bytes': [51], 'logprob': -0.1...",3,3,"[-0.16022726893424988, -1.9102272987365723, -1...","[0.851950095717818, 0.14804672334373317, 3.174...",0.604993,0.703903,2.576383,0.445534,0.703903,0.554466


In [39]:
print(type(df['original_label_map'].iloc[0]))
print(type(df['output'].iloc[0]))


<class 'numpy.int64'>
<class 'numpy.int64'>


In [40]:
(df['original_label_map'] != df['output']).sum()

468

In [41]:
N

60

In [42]:
(df.sort_values(by='entropy', ascending=False)['original_label_map'].head(N) != df.sort_values(by='entropy', ascending=False)['output'].head(N)).sum()

38

In [43]:
top_50_uncertain = df.sort_values(by='entropy', ascending=False).head(N)
top_50_uncertain

,text,output,logprobs,top_logprobs,original_label,original_label_map,logprobs_cal,probs,entropy,margin,kl_divergence,normalized_entropy,normalized_margin,normalized_kl_divergence
346,This is SO De Palma .,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.6...",3,3,"[-0.6718884110450745, -0.9218884110450745, -2....","[0.5107431631744972, 0.39776717542866474, 0.08...",1.357906,0.112976,2.054504,1.000000,0.112976,0.000000
307,Who is this movie for ?,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.5...",3,3,"[-0.5130981802940369, -1.0130982398986816, -3....","[0.5986380158549988, 0.36309228904360674, 0.03...",1.153993,0.235546,2.195846,0.849833,0.235546,0.150167
773,In an ART FILM !,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.3...",4,4,"[-0.3342668116092682, -1.8342667818069458, -2....","[0.7158627493186926, 0.15973057466011717, 0.12...",1.142118,0.556132,2.204076,0.841088,0.556132,0.158912
46,I am highly amused by the idea that we have co...,3,"{'content': [{'token': '3', 'bytes': [51], 'lo...","[{'token': '3', 'bytes': [51], 'logprob': -0.6...",5,5,"[-0.6931707262992859, -0.6931707262992859, -10...","[0.4999882384149322, 0.4999882384149322, 2.003...",1.000386,0.000000,2.302317,0.736713,0.000000,0.263287
4,The fourth in a series that I 'll bet most par...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.6...",3,3,"[-0.6931512355804443, -0.6931512355804443, -12...","[0.4999979641064766, 0.4999979641064766, 3.072...",1.000078,0.000000,2.302531,0.736486,0.000000,0.263514
618,It 's hard to imagine anyone managing to steal...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.6...",5,5,"[-0.6931473612785339, -0.6931473612785339, -16...","[0.49999994397329056, 0.49999994397329056, 9.2...",1.000003,0.000000,2.302583,0.736430,0.000000,0.263570
384,It all looks and plays like a $ 40 million ver...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.6...",2,2,"[-0.6931471824645996, -0.6931471824645996, -16...","[0.4999999684423957, 0.4999999684423957, 5.626...",1.000002,0.000000,2.302584,0.736429,0.000000,0.263571
905,It would be disingenuous to call Reno a great ...,2,"{'content': [{'token': '2', 'bytes': [50], 'lo...","[{'token': '2', 'bytes': [50], 'logprob': -0.6...",3,3,"[-0.6931471824645996, -0.6931471824645996, -17...","[0.49999998364959664, 0.49999998364959664, 2.3...",1.000001,0.000000,2.302584,0.736429,0.000000,0.263571
589,"No one involved , save Dash , shows the slight...",1,"{'content': [{'token': '1', 'bytes': [49], 'lo...","[{'token': '1', 'bytes': [49], 'logprob': -0.6...",1,1,"[-0.6931471824645996, -0.6931471824645996, -19...","[0.4999999976494762, 0.4999999976494762, 3.597...",1.000000,0.000000,2.302585,0.736428,0.000000,0.263572
509,This kind of hands-on storytelling is ultimate...,4,"{'content': [{'token': '4', 'bytes': [52], 'lo...","[{'token': '4', 'bytes': [52], 'logprob': -0.6...",4,4,"[-0.6931471824645996, -0.6931471824645996, -20...","[0.49999999784969984, 0.49999999784969984, 1.1...",1.000000,0.000000,2.302585,0.736428,0.000000,0.263572


In [44]:
top_50_uncertain = df.sort_values(by='entropy', ascending=False).head(N)[['text', 'original_label']].rename(columns={'original_label': 'label'})


In [45]:
top_50_uncertain

,text,label
346,This is SO De Palma .,3
307,Who is this movie for ?,3
773,In an ART FILM !,4
46,I am highly amused by the idea that we have co...,5
4,The fourth in a series that I 'll bet most par...,3
618,It 's hard to imagine anyone managing to steal...,5
384,It all looks and plays like a $ 40 million ver...,2
905,It would be disingenuous to call Reno a great ...,3
589,"No one involved , save Dash , shows the slight...",1
509,This kind of hands-on storytelling is ultimate...,4


In [46]:
# top_50_uncertain.to_csv(output_path, index=False)